In [44]:
import polars as pl
from pathlib import Path

# stats generated from ~1300 replays from 1/1/2023 - 4/30/2023
tech = pl.read_parquet(Path(R"./Output/techs.parquet"))
pl.Config.set_tbl_rows(-1)
tech.schema


{'date_time': Datetime(time_unit='us', time_zone='America/Chicago'),
 'slippi_version': Utf8,
 'match_id': Utf8,
 'match_type': Utf8,
 'game_number': Int64,
 'stage': Utf8,
 'duration': Duration(time_unit='ms'),
 'result': Utf8,
 'port': Utf8,
 'connect_code': Utf8,
 'character': Utf8,
 'costume': Utf8,
 'opnt_character': Utf8,
 'frame_index': Int64,
 'stocks_remaining': Int64,
 'tech_type': Utf8,
 'was_punished': Boolean,
 'position': List(Float64),
 'ground_id': Int64,
 'is_on_platform': Boolean,
 'is_missed_tech': Boolean,
 'towards_center': Boolean,
 'towards_opponent': Boolean,
 'jab_reset': Boolean,
 'last_hit_by': Utf8}

In [52]:
# Total occurances of each tech option
type_totals = tech.filter(pl.col("character") == "FALCO").groupby(pl.col("tech_type")).agg(
    pl.count()
).sort("count", descending=True)
missed_totals = tech.filter(pl.col("character") == "FALCO").groupby(pl.col("is_missed_tech")).agg(
    pl.count()
).sort("count", descending=True)
print(missed_totals)
print(type_totals)

shape: (2, 2)
┌────────────────┬───────┐
│ is_missed_tech ┆ count │
│ ---            ┆ ---   │
│ bool           ┆ u32   │
╞════════════════╪═══════╡
│ true           ┆ 5405  │
│ false          ┆ 5325  │
└────────────────┴───────┘
shape: (13, 2)
┌────────────────────────┬───────┐
│ tech_type              ┆ count │
│ ---                    ┆ ---   │
│ str                    ┆ u32   │
╞════════════════════════╪═══════╡
│ TECH_IN_PLACE          ┆ 2240  │
│ MISSED_TECH            ┆ 2004  │
│ TECH_LEFT              ┆ 1596  │
│ MISSED_TECH_GET_UP     ┆ 1520  │
│ MISSED_TECH_ROLL_LEFT  ┆ 913   │
│ TECH_RIGHT             ┆ 867   │
│ MISSED_TECH_ROLL_RIGHT ┆ 580   │
│ WALL_JUMP_TECH         ┆ 529   │
│ GET_UP_ATTACK          ┆ 266   │
│ JAB_RESET              ┆ 127   │
│ MISSED_WALL_TECH       ┆ 72    │
│ WALL_TECH              ┆ 13    │
│ MISSED_CEILING_TECH    ┆ 3     │
└────────────────────────┴───────┘


Seems I miss about 50% of all of my techs. Oof. It's not necessarily the end of the world, because intentionally missing a tech can be a mixup (e.g. vs sheik down throw). I happen to know for a fact though that I do just miss a lot of techs =(

It also looks like I favor tech in place, and almost never use getup attacks. Rolls left and right are roughly balanced, but that doesn't mean much on its own. A bit further down, we'll check if I'm teching towards my opponent, and/or towards center stage

My wall techs look pretty good, though it doesn't say much about how many I could have had via SDI + walltech (which I know I don't go for often enough)

In [6]:
# Total techs towards center, nulls are non-tech-rolls
tech.filter(pl.col("character") == "FALCO").groupby(pl.col("towards_center")).agg(pl.count()).sort(
    "count", descending=True
)

towards_center,count
bool,u32
null,6774
false,2374
true,1582


Seems I tech in place about twice as often as I tech roll, and I tech roll away from center about 60% more often than towards center. 

This makes sense as I'm very confident in my recovery and my ability to escape the corner, so I don't mind losing the positional advantage if it means I very likely won't get hit.

It can cause some problems though against very high level players who I can't consistently recover against or get out of the corner against. Even if I don't get directly punished for the tech, good players still position themselves well and don't hard commit when they realize I roll towards ledge often. It probably leads to a common feeling I've had where I can never quite "regain my footing", since I'm backed into a corner. Players who can consistently edgeguard me tend to roll me in less than 2 minutes as well, probably for this exact reason. I should probably work on incorporating more rolls in, even if I'm likely to get hit, so that I can hopefully SDI/DI myself to the relative safety of a platform or something.

In [7]:
# Total techs towards opponent, nulls are non-tech-rolls
tech.filter(pl.col("character") == "FALCO").groupby(pl.col("towards_opponent")).agg(pl.count()).sort(
    "count", descending=True
)

towards_opponent,count
bool,u32
null,6774
false,2195
true,1761


Similar ratios to towards_center, but I do end up rolling towards my opponent slightly more often than towards center. I wonder...

In [56]:
tech.filter(pl.col("character") == "FALCO").groupby(pl.col("towards_opponent", "towards_center")).agg(pl.count()).sort(
    "count", descending=True
)

towards_opponent,towards_center,count
bool,bool,u32
null,null,6774
false,false,1904
true,true,1291
true,false,470
false,true,291


The ratios of [towards/away from both] vs [towards one but not the other] makes sense, as it's more likely that your opponent will be closer to center than you after a hit. It looks like overall I favor rolling away from center in general, even if it's away from my opponent. I do this on purpose as an anti-meta pattern, since I know lots of players roll in predictably. As mentioned above though, this might be a more dangerous habit than it's worth against select players.

In [36]:
# Techs towards center by stocks remaining
tech.filter(pl.col("character") == "FALCO").groupby("stocks_remaining", "towards_center").agg(pl.count()).sort(
    "stocks_remaining"
)

stocks_remaining,towards_center,count
i64,bool,u32
1,true,309
1,false,413
1,null,1236
2,null,1682
2,true,351
2,false,601
3,false,637
3,true,443
3,null,1901


By looking at the statistics by stock, we can check if there are any noticeable patterns that only occur at specific phases in the game. Many players tend to increase their rolls towards center and away from their opponent as they get closer to losing, as a risk aversion measure.

| stocks| all techs | rolls
--- | --- | ---
| 4 | 15.15% | 39.85%
| 3 | 14.86% | 41.02%
| 2 | 13.33% | 36.86%
| 1 | 15.78% | 42.80%

It looks like the ratis are all pretty similar, though as always I favor rolling away from center. On my second to last stock though, I roll away noteably more. I wonder if that leads to a higher punish percentage...


In [41]:
tech.filter(pl.col("character") == "FALCO").groupby(pl.col("was_punished")).agg(pl.count())

was_punished,count
bool,u32
false,8649
true,2081


It looks like about 1/5 of my techs are directly punished during the tech animation. Next we can try filtering it by stocks remaining

In [40]:
tech.filter(pl.col("character") == "FALCO").groupby(pl.col("stocks_remaining", "was_punished")).agg(pl.count()).sort(pl.col("stocks_remaining"))

stocks_remaining,was_punished,count
i64,bool,u32
1,true,389
1,false,1569
2,true,524
2,false,2110
3,true,571
3,false,2410
4,true,597
4,false,2560


| stocks | punish % |
|---|---|
| 4 | 18.91% |
| 3 | 21.30% |
| 2 | 19.89% |
| 1 | 19.87% |

I get punished the least on stock 1, which makes sense. It's a fresh game, so it's likely it's a new opponent who doesn't know my habits. 

There's a small spike on my 3rd stock, which corresponds to a greater number of rolls in. This could be due to me getting "lazy" or auto piloting during stock 3. It could also be during a time when they've caught on to my usual patterns and I haven't yet started mixing differently